# Classificação de Imagens: Gatos vs. Cachorros com CNN

## Introdução
Neste notebook, construímos um classificador de imagens utilizando Redes Neurais Convolucionais (CNN) para distinguir entre gatos e cachorros. O projeto segue os seguintes passos:

1. Preparação dos dados
2. Construção do modelo CNN
3. Avaliação do modelo
4. Testes com imagens externas


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix


## 1. Preparação dos Dados

In [ ]:
# Diretórios
base_dir = '/mnt/data/titanic_cnn_dogs_vs_cats/data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Geradores de dados
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=False
)


## 2. Construção e Treinamento da CNN

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50
)


### Gráficos de Acurácia e Perda

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.figure()
plt.plot(epochs, acc, 'b', label='Acurácia de treino')
plt.plot(epochs, val_acc, 'r', label='Acurácia de validação')
plt.title('Acurácia por época')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'b', label='Perda de treino')
plt.plot(epochs, val_loss, 'r', label='Perda de validação')
plt.title('Perda por época')
plt.legend()

plt.show()


## 3. Avaliação do Modelo

In [ ]:
# Avaliação final
Y_pred = (model.predict(test_generator) > 0.5).astype(int)
Y_true = test_generator.classes

print(classification_report(Y_true, Y_pred, target_names=["Cat", "Dog"]))


## 4. Testes com Imagens Novas

In [ ]:
from tensorflow.keras.utils import load_img, img_to_array

def predict_image(image_path):
    img = load_img(image_path, target_size=(150, 150))
    img_tensor = img_to_array(img)/255.0
    img_tensor = np.expand_dims(img_tensor, axis=0)
    prediction = model.predict(img_tensor)
    plt.imshow(img)
    plt.axis('off')
    plt.title("Predicted: " + ("Dog" if prediction > 0.5 else "Cat"))
    plt.show()

# Exemplo:
# predict_image("/mnt/data/titanic_cnn_dogs_vs_cats/images/exemplo.jpg")


## Conclusão
O modelo treinado demonstrou bom desempenho na distinção entre cães e gatos, com resultados visuais coerentes. A seguir será elaborado o relatório final baseado nesse notebook.